In [4]:
from tensorflow import keras 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output
from tensorflow.keras import optimizers

In [5]:
#Metropolis algorithm: 2D lattice of N points with, minus edges, each point has n=4 neighbours 
def energy_delta_wrap_around(state, i, j):
    #i,j = (x,y) are the coordinates of the dipole we will flip, that we picked at random. Note, (0,0) is the bottom left
    #state is the current state of the system, which we will now flip
    #I am working in units of epsilon/boltzmann constant
    side_length=np.size(state[0,:])
    if i==0:
        left=state[side_length-1, j]
    else:
        left=state[i-1,j]
    if i==side_length-1:
        right=state[0,j]
    else:
        right=state[i+1,j]
    if j==0:
        bottom=state[i, side_length-1]
    else:
        bottom=state[i,j-1]
    if j==side_length-1:
        top=state[i, 0]
    else:
        top=state[i,j+1]
    energy_delta=2*state[i,j]*(top+bottom+right+left)
    return energy_delta 

def initial_state_generator(N):
    #N is the length of the square lattice
    state=np.zeros(shape=(N,N))
    for i in range(0, N):
        for j in range(0,N):
            p=np.random.uniform(0,1)
            if p<=1/2:
                state[i,j]=-1
            else:
                state[i,j]=1
    return state
def visualize(state):
    plt.imshow(state, cmap='gray', alpha=1, origin='lower')

def energy_of_state(state):
    side_length=int(np.size(state[0,:]))
    energy=0
    for i in range(0, side_length):
        for j in range(0,side_length):
            if i==0:
                left=state[side_length-1, j]
            else:
                left=state[i-1,j]
            if i==side_length-1:
                right=state[0,j]
            else:
                right=state[i+1,j]
            if j==0:
                bottom=state[i, side_length-1]
            else:
                bottom=state[i,j-1]
            if j==side_length-1:
                top=state[i, 0]
            else:
                top=state[i,j+1]
            energy+=-1*state[i,j]*(top+right+bottom+left)
    return energy/2 #compensates for double counting each energy 
def magnetization_of_state(state):
    side_length=int(np.size(state[0,:]))
    s=0
    for i in range(0, side_length):
        for j in range(0,side_length):
            s+=state[i,j]
    return s
    
            
def run_metropolis(N, T, dipole_flips_per_run, skip_animation_steps, show_plot=False):
    #N is the lattice dimension, T is the Temperature, show_plot controls whether we periodically plot the state
    #We cant simulate T=0
    #dipole_flips_per_run counts how many times we would like to flip each dipole in a run, very very roughly. 
    state=initial_state_generator(N)
    iter_range=dipole_flips_per_run*N*N
    S_state=np.zeros(iter_range)
    U_state=np.zeros(iter_range)
    for i in range(0,iter_range):
        S_state[i]=magnetization_of_state(state)
        U_state[i]=energy_of_state(state)
        flip_coords=(np.random.randint(low=0, high=N), np.random.randint(low=0, high=N))
        delta_energy=energy_delta_wrap_around(state, flip_coords[0], flip_coords[1])
        #Now we implement the actual logic of the Metropolis Algorithm
        if delta_energy<=0:
            #When the energy change is negative, always flip the given coordinate
            state[flip_coords[0], flip_coords[1]]=-1*state[flip_coords[0], flip_coords[1]]
        else:
            #When the energy is positive, calculate flip probability based on boltzmann factor. Change state accordingly
            p_flip=np.exp((-1*delta_energy)/T)
            if np.random.rand()<=p_flip:
                state[flip_coords[0], flip_coords[1]]=-1*state[flip_coords[0], flip_coords[1]]
        
        if show_plot and i%skip_animation_steps==0:
            clear_output(wait=True)
            plt.imshow(state, cmap='gray', alpha=1, origin='lower')
            plt.show()
    return state, U_state, S_state
            
def find_average(U_state):
    #Given a np array of all energies that are 'seen' in a metropolis run, this function returns the weighted
    #average of all energies 
    #Exactly the same function applies for the magnetization, except with s_state passed in
    u_max=int(np.amax(U_state))
    u_min=int(np.amin(U_state))
    frequencies=np.zeros(int(1+u_max-u_min))
    for i in range(u_min, u_max+1):
        for item in U_state:
            if item==i:
                frequencies[i-u_min]+=1
    weighted_sum=0
    for i in range(0, np.size(frequencies)):
        weighted_sum+=frequencies[i]*(u_min+i)
    avg=(1/np.size(U_state))*weighted_sum
    return avg



    
        
        
    
    
    
    
        
        
    

In [26]:
#Using the metropolis code above as generator of data samples, we will now attempt to teach a CNN to predict 
#average magnetization and energy. For Now, i will fix N, because I will pass in the final state of the network
#defining the data generating functions:
def generate_single_image_without_specifying_T(N):
    T=np.random.randint(1, 11)
    dipole_flips_per_run=100
    img, U, S= run_metropolis(N, T, dipole_flips_per_run, 1)
    training_result=np.array([find_average(U), find_average(S)])
    return img, training_result
def generate_batch_of_images_without_specifying_T(N, batchsize=30):
    batch_training_data=np.zeros(shape=(batchsize, N, N))
    batch_training_result=np.zeros(shape=(batchsize,2))
    for i in range(batchsize):
        batch_training_data[i,:,:],batch_training_result[i,:]=generate_single_image_without_specifying_T(N)
    return batch_training_data, batch_training_result
#Generate only average energies. We know from mean-field approach that magnetization and temperature aren't related.
#in a continuos fashion. So, it doesn't make much sense to have a network predict magnetization. 

def generate_single_sample_only_U_no_T(N):
    T=np.random.randint(1, 11)
    dipole_flips_per_run=100
    img, U, S= run_metropolis(N, T, dipole_flips_per_run, 1)
    training_result=find_average(U)
    return img, training_result

def generate_batch_only_U_no_T(N, batchsize=15):
    batch_training_data=np.zeros(shape=(batchsize, N, N))
    batch_training_result=np.zeros(batchsize)
    for i in range(batchsize):
        batch_training_data[i,:,:],batch_training_result[i]=generate_single_sample_only_U_no_T(N)
    return batch_training_data, batch_training_result

#Now, we try to tell the network the temperature at which the picture is taken. i.e. we will use a 2 channel input.
#Second channel indicates temperature. 

def generate_sample_U_and_T(N):#This function doesn't exactly generate a single sample because it returns T as a number
    #Use batchsize=1 in the next function to get a single sample
    T=np.random.randint(1, 11)
    dipole_flips_per_run=100
    img, U, S= run_metropolis(N, T, dipole_flips_per_run, 1)
    training_result=find_average(U)
    return img, T, training_result

def generate_batch_U_and_T(N, batchsize=15):
    batch_training_data=np.zeros(shape=(batchsize, N, N, 2))
    batch_training_result=np.zeros(batchsize)
    for i in range(batchsize):
        img, T, U=generate_sample_U_and_T(N)
        intermediate=np.zeros(shape=(N,N,2))
        intermediate[:,:,0]=img
        intermediate[:,:,1]=T
        batch_training_data[i,:,:,:]=intermediate
        batch_training_result[i]=U
    return batch_training_data, batch_training_result
    

    
        
        
    

In [7]:
#Defining the network
N=5
batchsize=15
Net=keras.models.Sequential()
Net.add(keras.layers.Conv2D(3,3,input_shape=(N,N,1), padding='same', activation='tanh'))
Net.add(keras.layers.AveragePooling2D(pool_size=(2,2), padding='same'))
Net.add(keras.layers.Conv2D(6,3, padding='same', activation='tanh'))
Net.add(keras.layers.AveragePooling2D(pool_size=(2,2), padding='same'))
Net.add(keras.layers.Conv2D(1,3, padding='same', activation='tanh'))
Net.add(keras.layers.Flatten())
Net.add(keras.layers.Dense(100, activation='tanh'))
Net.add(keras.layers.Dense(50, activation='tanh'))
Net.add(keras.layers.Dense(25, activation='softsign'))
Net.add(keras.layers.Dense(1,activation='linear'))
Net.compile(loss='mean_squared_error', optimizer='adam', )

2023-01-23 12:37:04.675724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [52]:
#Training the network in the simple case of only passing in final state. 
epochs=100
batchsize=15
N=5
costs=np.zeros(epochs)
for i in range(epochs):
    batch, result=generate_batch_only_U_no_T(N, batchsize=15)
    costs[i]=Net.train_on_batch(batch, result)
print('done!')


done!


In [39]:
#Now, we define a similar CNN, with the only change 
#that we add a channel that informs the network about the temperature
N=5
batchsize=15
NewNet=keras.models.Sequential()
NewNet.add(keras.layers.Conv2D(3,3,input_shape=(N,N,2), padding='same', activation='softsign'))
NewNet.add(keras.layers.AveragePooling2D(pool_size=(2,2), padding='same'))
NewNet.add(keras.layers.Conv2D(6,3, padding='same', activation='softsign'))
NewNet.add(keras.layers.AveragePooling2D(pool_size=(2,2), padding='same'))
NewNet.add(keras.layers.Conv2D(1,3, padding='same', activation='softsign'))
NewNet.add(keras.layers.Flatten())
NewNet.add(keras.layers.Dense(100, activation='tanh'))
NewNet.add(keras.layers.Dense(25, activation='softsign'))
NewNet.add(keras.layers.Dense(1))
NewNet.compile(loss='mean_squared_error', optimizer='adam', )

In [40]:
#training the network. We want it to predict the average energy, given final state and temperature at Ising model
epochs=100
batchsize=15
N=5
costs=np.zeros(epochs)
for i in range(epochs):
    batch, result=generate_batch_U_and_T(N, batchsize=15)
    costs[i]=NewNet.train_on_batch(batch, result)
print('done!')




done!
